# XGBoost
(C) 2018 Dariusz Kajtoch

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
data = pd.read_csv('./predict_blood_donations.csv')
print(data.columns)
data.columns = ['Id', 'MonthLast', 'Num', 'Vol', 'MonthFirst', 'Predict']
data.head()

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'Made Donation in March 2007'],
      dtype='object')


,Id,MonthLast,Num,Vol,MonthFirst,Predict
0,619,2,50,12500,98,1
1,664,0,13,3250,28,1
2,441,1,16,4000,35,1
3,160,2,20,5000,45,1
4,358,1,24,6000,77,0


# Features

In [6]:
X = data[[
    'MonthLast',
    'Num',
    'MonthFirst'
]]
#X['Ratio'] = X['MonthFirst']/X['MonthLast']
X['log'] = np.log(X['MonthFirst'] - X['MonthLast']+1.)

y = data['Predict']

/opt/intel/intelpython3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from xgboost import XGBClassifier

clf = XGBClassifier(n_estimators=350,
    random_state = 45,
    learning_rate=0.1,
    max_depth=1,
    gamma=0.,
    min_child_weight=1,
    subsample=0.5,
    colsample_bytree=1.,
#    scale_pos_weight,
    reg_alpha = 7.,
    reg_lambda = 2.5                   
)

skf = StratifiedKFold(n_splits=50, random_state=1234)


roc_tab = []; loss_tab = []; acc_tab = []
for train_index, test_index in skf.split(X,y):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test   = X.iloc[test_index], y.iloc[test_index]
    
    clf.fit(X_train, y_train)
    
    proba = clf.predict_proba(X_test)*0.9
    y_pred = clf.predict(X_test)
    
    roc_tab.append(roc_auc_score(y_test, proba[:,1]))
    loss_tab.append(log_loss(y_test, proba[:,1]))
    acc_tab.append(accuracy_score(y_test, y_pred))

/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/opt/intel/intelpython3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

In [8]:
print('AUC: %.8f +/- %.8f, min: %.8f' % (np.mean(roc_tab), np.std(roc_tab), np.min(roc_tab)))
print('LogLoss: %.8f +/- %.8f, max: %.8f' % (np.mean(loss_tab), np.std(loss_tab), np.max(loss_tab)))
print('Accuracy: %.8f +/- %.8f, min: %.8f' % (np.mean(acc_tab), np.std(acc_tab), np.min(acc_tab)))

AUC: 0.90652778 +/- 0.12927853, min: 0.44444444
LogLoss: 0.49504543 +/- 0.08383934, max: 0.68120992
Accuracy: 0.76900000 +/- 0.08628184, min: 0.50000000


In [208]:
from collections import OrderedDict

test = pd.read_csv('./test.csv')
print(test.columns.tolist())
X_test = test[[
    'Months since Last Donation',
    'Number of Donations',
    'Months since First Donation'
]]
X_test.columns = ['MonthLast', 'Num', 'MonthFirst']
proba = clf.predict_proba(X_test)[:,1]

submission = pd.DataFrame(
    OrderedDict([('',test['Unnamed: 0']), ('Made Donation in March 2007',proba*0.9)])
)

submission.to_csv('./submission.csv', index=False)

['Unnamed: 0', 'Months since Last Donation', 'Number of Donations', 'Total Volume Donated (c.c.)', 'Months since First Donation']


# Hyperparameter Tuning

In [52]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

est = XGBClassifier(random_state=1234)

param_grid = {
    'n_estimators': [150, 250],
    'reg_lambda': np.linspace(0,20,10),
    'reg_alpha': np.linspace(0,20,10)
}

clf = GridSearchCV(est,
    param_grid = param_grid,
    scoring = ['roc_auc', 'neg_log_loss'],
    refit = 'roc_auc',
    cv=10,
    verbose=100
)
clf.fit(X,y)

Fitting 10 folds for each of 200 candidates, totalling 2000 fits
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=0.0 .................
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=0.0, roc_auc=0.6915584415584415, neg_log_loss=-1.2332527591236706, total=   0.2s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=0.0 .................
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=0.0, roc_auc=0.8003246753246753, neg_log_loss=-0.5571615114037333, total=   0.1s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=0.0 .................
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=0.0, roc_auc=0.8522727272727272, neg_log_loss=-0.4279159601970479, total=   0.1s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=0.0 .................
[CV]  n_estimators

[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=4.444444444444445, roc_auc=0.930232558139535, neg_log_loss=-0.36615013111648814, total=   0.1s
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    5.7s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=4.444444444444445 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=4.444444444444445, roc_auc=0.5268336314847943, neg_log_loss=-0.672176473075524, total=   0.1s
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    5.8s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=6.666666666666667 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=6.666666666666667, roc_auc=0.6883116883116882, neg_log_loss=-0.9600356300843174, total=   0.1s
[Parallel(n_jobs=1)]: Done  31 out of  31 | elapsed:    6.0s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=6.666666666666667 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=6.666666666666667, roc_auc=0.8376623376623377, neg_log_loss=-0.

[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111, roc_auc=0.523538961038961, neg_log_loss=-0.9029793324077438, total=   0.1s
[Parallel(n_jobs=1)]: Done  57 out of  57 | elapsed:   10.5s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111, roc_auc=0.8538961038961039, neg_log_loss=-0.4930476216149741, total=   0.1s
[Parallel(n_jobs=1)]: Done  58 out of  58 | elapsed:   10.7s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111, roc_auc=0.9427549194991055, neg_log_loss=-0.3693126275071076, total=   0.1s
[Parallel(n_jobs=1)]: Done  59 out of  59 | elapsed:   10.8s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=11.11111111111111, roc_auc=0.5161001788908766, neg_log_loss=-0.

[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778, roc_auc=0.6801948051948051, neg_log_loss=-0.5209165898890331, total=   0.1s
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed:   15.4s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778, roc_auc=0.42207792207792205, neg_log_loss=-0.6781588234007359, total=   0.1s
[Parallel(n_jobs=1)]: Done  86 out of  86 | elapsed:   15.5s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778, roc_auc=0.5121753246753247, neg_log_loss=-0.9094919168615135, total=   0.1s
[Parallel(n_jobs=1)]: Done  87 out of  87 | elapsed:   15.7s remaining:    0.0s
[CV] n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778 ...
[CV]  n_estimators=150, reg_alpha=0.0, reg_lambda=17.77777777777778, roc_auc=0.8636363636363636, neg_log_loss=-

[CV]  n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=2.2222222222222223, roc_auc=0.9570661896243292, neg_log_loss=-0.3744580624625087, total=   0.1s
[CV] n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=2.2222222222222223 
[CV]  n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=2.2222222222222223, roc_auc=0.5590339892665473, neg_log_loss=-0.6286299825379891, total=   0.1s
[CV] n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=4.444444444444445 
[CV]  n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=4.444444444444445, roc_auc=0.7021103896103896, neg_log_loss=-0.8144596339813595, total=   0.1s
[CV] n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=4.444444444444445 
[CV]  n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=4.444444444444445, roc_auc=0.900974025974026, neg_log_loss=-0.48968900097855206, total=   0.1s
[CV] n_estimators=150, reg_alpha=2.2222222222222223, reg_lambda=4.444444444444445 
[CV]  n_estimators=150, r

...

[CV]  n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=13.333333333333334, roc_auc=0.774597495527728, neg_log_loss=-0.5037352315017155, total=   0.1s
[CV] n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557 
[CV]  n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557, roc_auc=0.6842532467532467, neg_log_loss=-0.5897634019111765, total=   0.1s
[CV] n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557 
[CV]  n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557, roc_auc=0.6168831168831168, neg_log_loss=-0.5806148052215576, total=   0.2s
[CV] n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557 
[CV]  n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557, roc_auc=0.7637987012987013, neg_log_loss=-0.5218517762319795, total=   0.1s
[CV] n_estimators=250, reg_alpha=17.77777777777778, reg_lambda=15.555555555555557 
[CV]  n_estimators=250, reg_a

[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=0.0, roc_auc=0.8125, neg_log_loss=-0.5254118624946167, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=0.0 ................
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=0.0, roc_auc=0.4788961038961039, neg_log_loss=-0.6184899534644752, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=0.0 ................
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=0.0, roc_auc=0.7556818181818181, neg_log_loss=-0.5272986187503256, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=0.0 ................
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=0.0, roc_auc=0.8783542039355994, neg_log_loss=-0.46395355675901684, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=0.0 ................
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=0.0, roc_auc=0.774597495527728, neg_log_loss=-0.5021306883011546, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=2.2222222222222223

[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889, roc_auc=0.8125, neg_log_loss=-0.5237914652659975, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889 ...
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889, roc_auc=0.4788961038961039, neg_log_loss=-0.6201267948952215, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889 ...
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889, roc_auc=0.7540584415584417, neg_log_loss=-0.5276930488902947, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889 ...
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889, roc_auc=0.8828264758497317, neg_log_loss=-0.4643539142395769, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889 ...
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=8.88888888888889, roc_auc=0.7558139534883721, neg_log_loss=-0.5015806329569646, total=   0.2s
[CV

[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778, roc_auc=0.8538961038961039, neg_log_loss=-0.477136018204278, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778 ..
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778, roc_auc=0.7849025974025974, neg_log_loss=-0.49733417969325494, total=   0.2s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778 ..
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778, roc_auc=0.8125, neg_log_loss=-0.5230960681520659, total=   0.2s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778 ..
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778, roc_auc=0.4821428571428571, neg_log_loss=-0.6136895531210406, total=   0.1s
[CV] n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778 ..
[CV]  n_estimators=250, reg_alpha=20.0, reg_lambda=17.77777777777778, roc_auc=0.752435064935065, neg_log_loss=-0.5276239704469154, total=   0.1s

GridSearchCV(cv=10, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [150, 250], 'reg_lambda': array([ 0.     ,  2.22222,  4.44444,  6.66667,  8.88889, 11.11111,
       13.33333, 15.55556, 17.77778, 20.     ]), 'reg_alpha': array([ 0.     ,  2.22222,  4.44444,  6.66667,  8.88889, 11.11111,
       13.33333, 15.55556, 17.77778, 20.     ])},
       pre_dispatch='2*n_jobs', refit='roc_auc', return_train_score='warn',
       scoring=['roc_auc', 'neg_log_loss'], verbose=100)

In [54]:
print(clf.best_score_)
print(clf.best_params_)

0.7926428182605508
{'n_estimators': 150, 'reg_alpha': 6.666666666666667, 'reg_lambda': 2.2222222222222223}


In [60]:
print(clf.cv_results_['params'][0])

{'n_estimators': 150, 'reg_alpha': 0.0, 'reg_lambda': 0.0}
